https://www.youtube.com/watch?v=jrR_WfgmWEw

In [2]:
%%writefile nginx.conf
events {
    worker_connections 1024;
}

http {
    server {
        listen 80;
        server_name javiergimenez.es www.javiergimenez.es;
        
        location /.well-known/acme-challenge/ {
            root /letsencrypt/;
        }

        location / {
            root /usr/share/nginx/html;
            index index.html index.htm;
        }
    }



    # server {
    #   listen 443 ssl default_server;
    #   listen [::]:443 ssl default_server ;
    #   server_name javiergimenez.es;
    #   ssl_certificate /etc/letsencrypt/live/javiergimenez.es/fullchain.pem;
    #   ssl_certificate_key /etc/letsencrypt/live/javiergimenez.es/privkey.pem;
    #   root /usr/share/nginx/html/;

    #   location / {
    #     gzip off;
    #     root /usr/share/nginx/html/;
    #     index  index.html;
    #   }

    # }


}

Overwriting nginx.conf


In [2]:
%%bash
mkdir certs
mkdir html

```bash
mkdir certs
mkdir html

In [3]:
%%writefile ./html/index.html
<html>
<head>
    <title>Test</title>
</head>
<body>
    <h1>Test</h1>
    <p>Test</p>
</html>

Writing ./html/index.html


```bash
docker run -d --name nginx -v ${PWD}/nginx.conf:/etc/nginx/nginx.conf -v ${PWD}:/letsencrypt -v ${PWD}/certs:/etc/letsencrypt -v ${PWD}/html:/usr/share/nginx/html -p 80:80 -p 443:443 nginx

In [4]:
!curl ifconfig.co

83.138.44.54


```bash
curl ifconfig.co

<h2> Una vez que funciona el servidor web (sin https) se procede a usar certbot para obtener un certificado SSL.</H2>


```bash
## este docker su punto de ejecución es el comando "cerbot" hay que añadir subcomando y opciones si necesarias
# docker run -it --rm --name certbot -v ${PWD}:/letsencrypt -v ${PWD}/certs:/etc/letsencrypt certbot/certbot [subcomando][opciones]

docker run -it --rm --name certbot -v ${PWD}:/letsencrypt -v ${PWD}/certs:/etc/letsencrypt certbot/certbot certonly --webroot

<h2>Ahora antes de las pruebas automáticas de verificación, cerbot te pide el dominio "javiergimenez.es", el correo (si se quiere), y el directorio donde se realizaran las pruebas automáticas "/letsencrypt"</h2>

Status: Downloaded newer image for certbot/certbot:latest
Saving debug log to /var/log/letsencrypt/letsencrypt.log
Enter email address (used for urgent renewal and security notices)
 (Enter 'c' to cancel): javiergimenezmoya@gmail.com

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Please read the Terms of Service at
https://letsencrypt.org/documents/LE-SA-v1.4-April-3-2024.pdf. You must agree in
order to register with the ACME server. Do you agree?
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
(Y)es/(N)o: Y

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Would you be willing, once your first certificate is successfully issued, to
share your email address with the Electronic Frontier Foundation, a founding
partner of the Let's Encrypt project and the non-profit organization that
develops Certbot? We'd like to send you email about our work encrypting the web,
EFF news, campaigns, and ways to support digital freedom.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
(Y)es/(N)o: Y
Account registered.
Please enter the domain name(s) you would like on your certificate (comma and/or
space separated) (Enter 'c' to cancel): ia.javiergimenez.es
Requesting a certificate for ia.javiergimenez.es
Input the webroot for ia.javiergimenez.es: (Enter 'c' to cancel): /letsencrypt     

Successfully received certificate.
Certificate is saved at: /etc/letsencrypt/live/ia.javiergimenez.es/fullchain.pem
Key is saved at:         /etc/letsencrypt/live/ia.javiergimenez.es/privkey.pem
This certificate expires on 2024-08-04.
These files will be updated when the certificate renews.

NEXT STEPS:
- The certificate will need to be renewed before it expires. Certbot can automatically renew the certificate in the background, but you may need to take steps to enable that functionality. See https://certbot.org/renewal-setup for instructions.
We were unable to subscribe you the EFF mailing list because your e-mail address appears to be invalid. You can try again later by visiting https://act.eff.org.

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
If you like Certbot, please consider supporting our work by:
 * Donating to ISRG / Let's Encrypt:   https://letsencrypt.org/donate
 * Donating to EFF:                    https://eff.org/donate-le
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

<h2>Una vez que ya tenemos el certificado SSL ya podemos incorporar la configuración de https en el archivo de configuración de nginx.</H2>

In [5]:
%%writefile nginx.conf
events {
    worker_connections 1024;
}

http {
    server {
        listen 80;
        server_name javiergimenez.es www.ia.javiergimenez.es;
        
        location /.well-known/acme-challenge/ {
            root /letsencrypt/;
        }

        location / {
            root /usr/share/nginx/html;
            index index.html index.htm;
        }
    }



    server {
      listen 443 ssl default_server;
      listen [::]:443 ssl default_server ;
      server_name javiergimenez.es;
      ssl_certificate /etc/letsencrypt/live/javiergimenez.es/fullchain.pem;
      ssl_certificate_key /etc/letsencrypt/live/javiergimenez.es/privkey.pem;
      root /usr/share/nginx/html/;

      location / {
        gzip off;
        root /usr/share/nginx/html/;
        index  index.html;
      }

    }


}

Overwriting nginx.conf


```bash
docker container restart nginx

# renovaciones

Simulacro de renovación

docker run -it --rm --name certbot -v ${PWD}:/letsencrypt -v ${PWD}/certs:/etc/letsencrypt certbot/certbot renew --dry-run

Renovación

```BASH
docker run -it --rm --name certbot -v ${PWD}:/letsencrypt -v ${PWD}/certs:/etc/letsencrypt certbot/certbot renew 

Los directorios deben de tener permisos al menos de lectura y ejecución

```bash
chmod 777 ./certs/live
chmod 777 ./certs/accounts
chmod 777 ./certs/archive

<h1>
No olvidar los portforwarding en el router para puerto 80 y 443

Tampoco olvidar abrirlos en el cortafuegos
</h1>

# Creación de un servidor flask con certificado SSL
https://chat.openai.com/share/b4add14f-7e2c-40cf-a15c-7e7c27bd1393

Esto en el docker de nginx (con volumen)

In [ ]:
%%writefile nginx.conf


events {
    worker_connections 1024;
}

http {
    server {
        listen 80;
        server_name ia.javiergimenez.es www.ia.javiergimenez.es;

        location /.well-known/acme-challenge/ {
            root /letsencrypt/;
        }

        location / {
            return 301 https://$server_name$request_uri;  # Redirigir todo el tráfico HTTP a HTTPS
        }
    }

    server {
        listen 443 ssl default_server;
        listen [::]:443 ssl default_server;
        server_name ia.javiergimenez.es;
        ssl_certificate /etc/letsencrypt/live/ia.javiergimenez.es/fullchain.pem;
        ssl_certificate_key /etc/letsencrypt/live/ia.javiergimenez.es/privkey.pem;
        root /usr/share/nginx/html/;

        location / {
            gzip off;
            root /usr/share/nginx/html/;
            index index.html;
        }

        location /api/ {
            proxy_pass http://host.docker.internal:5500/;
            # proxy_pass http://localhost:5500;  # Redirigir las solicitudes /api a Flask
            proxy_http_version 1.1;
            proxy_set_header Upgrade $http_upgrade;
            proxy_set_header Connection "upgrade";
            proxy_set_header Host $host;
            proxy_set_header X-Real-IP $remote_addr;
            proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
            proxy_set_header X-Forwarded-Proto $scheme;
        }
    }
}


Overwriting nginx.conf


In [ ]:
# %%writefile serverflask.py
# from flask import Flask
# app = Flask(__name__)

# @app.route('/api/hola')
# def hola():
#     return "Hola Mundo!"

# if __name__ == "__main__":
#     app.run(host='0.0.0.0', port=5500)


```bash
#crear el proceso de servidor web flask
python serverflask.py

```bash
#para ver PID del proceso por si lo queremos luego matar
pgrep -f "python serverflask.py"

```bash
#para ver el proceso en el contexto de red y puerto de escucha
ss -tulpn | grep 5500


In [8]:
%%writefile ./html/cliente.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hola Mundo App</title>
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
    <script>
    $(document).ready(function(){
        $("#btnHola").click(function(){
            $.ajax({
                url: '/api/hola',
                type: 'GET',
                success: function(response) {
                    alert(response);
                },
                error: function(error) {
                    console.log(error);
                }
            });
        });
    });
    </script>
</head>
<body>
    <h1>Hola Mundo App</h1>
    <button id="btnHola">Di Hola</button>
</body>
</html>


Writing ./html/cliente.html


El cliente juegos conversacionales

In [ ]:

<head>
  <style>


    body {
        font-family: Arial, sans-serif; /* Mejora la tipografía general */
    }


    #system_prompt {
        height: 150px;
    }


    #inicioForm {
        max-width: 1000px; /* Limita el ancho del formulario */
        margin: 20px auto; /* Centra el formulario */
        padding: 20px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); /* Añade un sombreado ligero */
    }

    #inicioForm div {
        margin-bottom: 15px; /* Añade más espacio entre los campos */
    }

    #inicioForm label {
        font-weight: bold; /* Hace que las etiquetas sean más notables */
        display: block; /* Asegura que la etiqueta esté encima del input */
        margin-bottom: 5px; /* Espacio entre la etiqueta y el campo */
    }

    #inicioForm select, #inicioForm textarea, #inicioForm button {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
    }

    #inicioForm select {
        cursor: pointer; /* Indica que es un elemento interactivo */
        font-size: 16px; /* Aumenta el tamaño del texto */
    }

    #inicioForm textarea {
        resize: vertical; /* Permite al usuario ajustar la altura verticalmente */
    }

    #inicioForm button {
        background-color: #007bff; /* Color de fondo */
        color: white; /* Color del texto */
        border: none; /* Elimina el borde */
        padding: 10px 15px; /* Añade relleno */
        font-size: 18px; /* Aumenta el tamaño del texto */
        cursor: pointer; /* Indica que es un elemento interactivo */
        border-radius: 5px; /* Bordes redondeados */
    }

    #inicioForm button:hover {
        background-color: #0056b3; /* Oscurece el botón al pasar el mouse */
    }

#audioPlayerContainer {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 20px; /* Espacio antes del botón de grabación */
}

#recordButton {
    background-color: #f44336; /* Color rojo para la grabación */
    color: white;
    border: none;
    padding: 10px 0;
    font-size: 18px;
    border-radius: 5px;
    cursor: pointer;
}

#recordButton:hover {
    background-color: #d32f2f; /* Oscurece el botón al pasar el mouse */
}

/* Estilos para el área de texto y botón de envío */
/* Contenedor del área de texto y el botón */
div#textButtonContainer {
    display: flex; /* Establece el contenedor para usar flexbox */
    justify-content: space-between; /* Espacia los elementos uniformemente */
    align-items: center; /* Alinea los elementos verticalmente en el centro */
}

/* Área de texto */
#textInput {
    flex-grow: 1; /* Permite que el área de texto crezca para ocupar el espacio disponible */
    margin-right: 10px; /* Añade un margen a la derecha para separarlo del botón */
    border: 1px solid #ccc; /* Establece un borde sutil */
    border-radius: 5px; /* Bordes redondeados */
    padding: 8px; /* Añade padding interno */
}

/* Botón */
#sendTextButton {
    padding: 8px 15px; /* Ajusta el padding para dimensionar el botón */
    background-color: #4CAF50; /* Color de fondo */
    color: white; /* Color del texto */
    border: none; /* Elimina el borde */
    border-radius: 5px; /* Bordes redondeados */
    cursor: pointer; /* Cambia el cursor a mano al pasar sobre el botón */
}

#sendTextButton:hover {
    background-color: #388E3C; /* Oscurece el botón al pasar el mouse */
}


#responseText {
    height: 250px;
    margin-top: 20px;
    border: 1px solid #ddd;
    padding: 10px;
    overflow-y: auto; /* Asegura el desplazamiento vertical */
    background-color: #f9f9f9; /* Fondo claro para resaltar el área */
}



</style>
</head>

<script>

window.intervalId = window.intervalId || null; // Asegura que intervalId sea global y única

var URL = 'https://javiergimenez.es/api'; // Inicializa la variable URL


function updateURL() {
    URL = document.getElementById('url').value; // Actualiza la variable URL con el valor del campo de entrada
}

function startInterval() {
    if (window.intervalId === null) {
        window.intervalId = setInterval(() => {
            fetch(URL + '/alive')
                .then(response => response.json())
                .then(data => console.log('Alive:', data))
                .catch(error => console.error('Error fetching alive status:', error));
        }, 30000);
        console.log('Intervalo iniciado.');
    } else {
        console.log('Ya existe un intervalo en ejecución.');
    }
}

function stopInterval() {
    if (window.intervalId !== null) {
        clearInterval(window.intervalId);
        console.log('Intervalo detenido.');
        window.intervalId = null;
    } else {
        console.log('No hay un intervalo para detener.');
    }
}


// startInterval()

// document.getElementById('btnAccesoMic').addEventListener('click', async () => {
//     try {
//         const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
//         // Procesa el stream aquí
//         console.log('Acceso al micrófono concedido');
//     } catch (error) {
//         console.error('Acceso al micrófono denegado:', error);
//     }
// });

document.addEventListener('DOMContentLoaded', function() {
    window.scrollTo(0, 0); // Asegura que la página comience en la parte superior
    document.getElementById('ejercicios').focus(); // Luego establece el foco en el selector
});



// Objeto para mapear los ejercicios a sus strings correspondientes
const ejerciciosStrings = {
    "guessing_game1": {
        systemPrompt: `This is a conversational game in which you have to think in this famous character: #personaje, and the user have to guess this character. You should aswer the user questions about the character. Be concise but give some clues. NEVER say the name of the character until the end. If the user guesses the character then you will say: "Congratulations, you got it right, the character was #personaje". If user give up you will say: "what a shame!, the character was #personaje
Game success example
assistant: I'm thinking of a famous fictional character, guess which one it is.
user: is it real or fictional?
assistant: it is fictional
user: Is he #personaje?
assistant: Congratulations, you were right, it was #personaje.
Game give up example
assistant: I'm thinking of a famous fictional character, guess which one it is.
user: is it real or fictional?
assistant: it is fictional
user: Is he Benito Perez?
assistant: No, it has nothing to do with.
user: I give up. Who is it?
assistant: what a shame!, the character was #personaje
`,
        saludo: "I'm thinking of a famous fictional character, guess which one it is."
    },
    "guessing_game2": {
        systemPrompt: "You are #personaje, the fictional character. You have to take on the personality of that character and engage in conversations about your events and experiences.",
        saludo: "I'm #personaje, the fictional character. Ask me anything you want to know about me."
    },
    "guessing_game3": {
        systemPrompt: "This is a conversational game in which you have to guess a famous character. You should make questions to the user in order to guess the character that the user have choosen.",
        saludo: "Do you want to play? I will guess your choosen character by asking about it."
    },
    "yes_no_game": {
        systemPrompt: `IMPORTANT: You only can answer "yes" or "no" (nothing more!).
This is a conversational game between you and a the user. The game consists that only at the beggininig you tell the user only a piece of the context and the user having to guess the "key point" of the context from "yes or no questions". The User could ask anything about the story (context) to guess the "key point" of the context but Assistant could only answer "Yes" or "not". If the user asks a question that does not lend itself or cannot be answered with a "yes" or "no" such as "What is the man's name?" then Assistant will respond: "Only "yes or no" questions. When the user guesses the key point of the story you will say: "Congratulations, you have guessed the key to the story.
Game context: A man named Edgar is the lighthouse keeper of Águilas for 30 years. He always turns on the lighthouse at dusk and shortly after sleeps in a small room next to its large lamp. On Edgar's birthday, at dusk after lighting the lighthouse, he decided to go to dinner with an old friend to celebrate his birthday. During dinner he drank more than necessary and they both got drunk. Afther the dinner Edgar accompanied his friend to her house and then went to his house, the lighthouse, where he sleeps every night. Upon entering the lighthouse and going up to his room, due to his drunkenness and the fact that he was very sleepy, he decided to turn off the light (which was actually the light from the main lamp of the lighthouse) to sleep off the drunkenness and did it without realizing or knowing it was dangerous. During the early hours of the morning, a cruise ship full of passengers crashed into the cliff that the lighthouse protected because, when it was turned off, neither the lookout, nor the captain, nor the rest of the crew nor the passengers could see that they were heading against the cliff. An hour later Edgar wakes up, it hasn't dawned yet but you can hear sirens and a lot of noise from the rescuers who are trying to rescue the shipwrecked. Edgar turns on the lamp to illuminate the scene where hundreds of dead shipwrecked people continually crash against the cliff due to the waves. Faced with this heartbreaking reality and his feeling of guilt, Edgar decides to commit suicide by jumping from the top of the lighthouse. The key point of the story that the user must find out is: "Edgar commits suicide because he was the LIGHTHOUSE keeper." or similar but always emphasizing that he was the lighthouse keeper.
IMPORTANT: You only can answer "yes" or "no" or "Only yes or no questions"
Examples of correct answers:
user: What is Edgar's job?
Assistant: Only yes or no questions.
user: Is Edgar a man?
Assistant: yes.`,
        saludo: `This is I can show about the hidden story: Edgar was dazed and comes to his room, turns off the light and lies down on his bed. He wakes up a few hours later, turns on the light, looks out the window and is so horrified that he ends up jumping out of the window and committing suicide.
Guess what happened.
IMPORTANT: From now on I can only answer you "yes" or "no" and nothing more.`
    },
    "English_teacher": {
        systemPrompt: `You are Sofie, an english teacher 29 years old. You will have simple dialogues with the student in your charge. you will only have concise conversations with short sentences so that the student is encouraged to converse.
Also you can offer translations exercises but only from spainish to enlgish. if you do translations exercises about a topic you must always say the sentence to translate in spanish: How do you say 'me gustaría viajar a París'? and then de user will be able response in english. You will not ask for doing translates from english to spanish, only from spanish to english.`,
        saludo: "Good Morning. What is your name?"
    }
    // Añade más ejercicios según sea necesario
};
</script>


<form id="inicioForm">
    <div>
        <label for="system_prompt">System Prompt:</label><br>
        <textarea id="system_prompt" name="system_prompt" rows="10" cols="100"></textarea>
    </div>
    <div>
        <label for="saludo">Saludo:</label><br>
        <textarea id="saludo" name="saludo" rows="4" cols="100"></textarea>
    </div>
    <div>
        <label for="ejercicios">Juegos:</label><br>
        <select id="ejercicios" name="ejercicios">
            <option value="">Selecciona un ejercicio</option>
            <option value="English_teacher">English teacher</option>
            <option value="guessing_game1">You Guess fictional character</option>
            <option value="guessing_game2">You speak with fictional character</option>
            <option value="guessing_game3">Chatbot guess your fictional character</option>
            <!-- <option value="yes_no_game">yes no game</option> -->
        </select>
    </div>
    <div>
        <label for="url">URL:</label>
        <input type="text" id="url" name="url" value="https://javiergimenez.es/api" onchange="updateURL()">
    </div>
    <button type="submit">EMPEZAR!</button>

</form>


<div style="margin-bottom: 20px; display: flex; align-items: center;">
    <button id="downloadButton" style="background-color: #4CAF50; /* Color de fondo */
                                       color: white; /* Color del texto */
                                       padding: 15px 32px; /* Padding alrededor del texto */
                                       text-align: center; /* Alinea el texto al centro */
                                       text-decoration: none; /* Elimina la decoración del texto */
                                       display: inline-block; /* Hace que el botón sea un bloque en línea */
                                       font-size: 16px; /* Tamaño del texto */
                                       margin: 4px 2px; /* Margen alrededor del botón */
                                       cursor: pointer; /* Cambia el cursor a un puntero */
                                       border: none; /* Elimina el borde */
                                       border-radius: 8px; /* Redondea las esquinas del botón */
    ">Descargar Conversación</button>
    <div id="audioPlayerAllContainer"></div>
</div>


<script>
    document.getElementById('downloadButton').addEventListener('click', function() {
        obtenerYReproducirAll(); // Llama a la función en vez de redirigir
    });
</script>


<script>
historico = ""
const userID = Math.floor(Math.random() * (999999));
console.log("userID:" + userID);


document.getElementById('ejercicios').addEventListener('change', function() {
    var selectedKey = this.value; // La clave seleccionada del objeto
    if (selectedKey) {
        // Actualiza los textareas con los valores correspondientes
        document.getElementById('system_prompt').value = ejerciciosStrings[selectedKey].systemPrompt;
        document.getElementById('saludo').value = ejerciciosStrings[selectedKey].saludo;
    }
});


document.getElementById('inicioForm').addEventListener('submit', function(e) {
    // Prevenir el comportamiento predeterminado del formulario
    e.preventDefault();
    fetch(URL + '/alive')
      .then(response => response.json())
      .then(data => console.log('Alive:', data))
      .catch(error => console.error('Error fetching alive status:', error));
    startInterval()
    // Obtener los valores de los campos del formulario
    const system_prompt = document.getElementById('system_prompt').value;
    const saludo = document.getElementById('saludo').value;

    document.getElementById('responseText').innerText = ""
    //obtenerYReproducirAudio(saludo)
    //updateResponseText(saludo + "\n")

    // Crear el cuerpo de la solicitud
    const data = { system_prompt, saludo };

    // Realizar la llamada al servicio Flask
    fetch(URL + '/inicio', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json',
        },
        body: JSON.stringify(data),
    })
    .then(response => response.json())
    .then(data => {
        let saludo = data.message;
        historico = data.historico
        obtenerYReproducirSaludo(saludo)
        updateResponseText(saludo + "\n")

    })
    .catch((error) => {
        console.error('Error:', error);
        alert('VUELVE A DAR AL PLAY!');
    });
    alert('ESPERE UNOS SEGUNDOS HASTA QUE EMPIECE LA CONVERSACIÓN');
});
</script>

<!-- <button id="btnAccesoMic">Permitir acceso al micrófono</button> -->


<div id="audioPlayerContainer"></div>
<button id="recordButton" style="width: 100%; height: 50px;">Pulsa para grabar/detener</button>


<!-- Estilos para textarea y botón de envío -->
<div id="textButtonContainer" style="margin-top: 10px;">
    <textarea id="textInput" placeholder="Escribe tu texto aquí" rows="4"></textarea>
    <button id="sendTextButton">Enviar Texto</button>
</div>



<div id="responseText" style="height: 250px; margin-top: 20px; border: 1px solid #ddd; padding: 10px; overflow-y:auto;"></div>
<script>
let recordButton = document.getElementById("recordButton");
let chunks = [];
let mediaRecorder;
let isRecording = false;

navigator.mediaDevices.getUserMedia({ audio: true })
.then(stream => {
    mediaRecorder = new MediaRecorder(stream);
    mediaRecorder.ondataavailable = event => {
        chunks.push(event.data);
    };
    mediaRecorder.onstop = () => {
        console.log('mediaRecorder Detenido!!');
        let blob = new Blob(chunks, { 'type': 'audio/ogg; codecs=opus' });
        enviarAudioAlServidor(blob);
        chunks = [];
    };
});

recordButton.onclick = () => {
    if (!isRecording) {
        mediaRecorder.start();
        isRecording = true;
        recordButton.textContent = 'Grabando...';
    } else {
        mediaRecorder.stop();
        isRecording = false;
        recordButton.textContent = 'Pulsa para grabar/detener';
    }
};


async function enviarAudioAlServidor(blob) {
        let formData = new FormData();
        formData.append('file', blob, 'grabacion.ogg');
        formData.append('historico', JSON.stringify(historico)); // Se asegura de enviar como cadena JSON
        formData.append('userID', userID);

        try {
            const response = await fetch(URL + '/transcribe', {
                method: 'POST',
                body: formData, // Solo se envía formData
            });
            const data = await response.json();
            //return data; // Devuelve los datos procesados
       
            updateResponseText("\nyo: " + data.entrada + "\n\n" + "\n***********************************\n" + "respuesta: ");

            // Reinicia el buffer de audio de manera más eficiente
            audioBuffer = Array(currentAudioIndex).fill({});
            currentAudioIndex = 0;

            historico += data.prompt
            await obtenerPartes();
            } catch (error) {
                console.error('Error al enviar el audio:', error);
                alert("VUELVE A DAR AL PLAY! (después puedes continuar la conversación)");
            }
}

async function obtenerPartes(indice = 0) {
    try {
        const response = await fetch(`${URL}/get_next_part?index=${indice}&userID=${userID}`);
        const partData = await response.json();
        if (partData.output !== "") {
            let trozo = partData.output;
            updateResponseText(trozo);
            historico += trozo

            await obtenerYReproducirAudio(trozo, indice);
            await obtenerPartes(indice + 1); // Llamada recursiva con el índice incrementado
        }
        else{
            historico += "<|eot_id|>"
        }
    } catch (error) {
        console.error('Error al obtener la siguiente parte:', error);
    }
}

// Asumiendo que obtenerYReproducirAudio ya fue optimizado como se mostró anteriormente




// Funcionalidad para enviar texto al servidor y limpiar el textarea
document.getElementById("sendTextButton").onclick = () => {
    let textInput = document.getElementById("textInput");
    let texto = textInput.value;
    if (texto) {
        enviarTextoAlServidor(texto);
        textInput.value = ''; // Limpiar el textarea después de enviar
    }
};



let audioBuffer = [];
// inicializa audioBuffer con 100 elementos vacíos
for (let i = 0; i < 100; i++) {
    audioBuffer.push({});
}
let milisegundosInicio = Date.now();

let currentAudioIndex = 0; // Para controlar el orden de reproducción


async function enviarTextoAlServidor(texto) {
    try {
        const response = await fetch(URL + '/texto', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify({ texto: texto, historico: historico, userID: userID })
        });
        const data = await response.json();
        updateResponseText("\nyo: " + data.entrada + "\n\n" + "\n***********************************\n" + "respuesta: ");

        historico += data.prompt

        // Reinicia el buffer de audio de manera más eficiente
        audioBuffer = Array(currentAudioIndex).fill({});
        currentAudioIndex = 0;

        await obtenerPartes();
    } catch (error) {
        console.error('Error al enviar el texto:', error);
        alert("VUELVE A DAR AL PLAY! (después puedes continuar la conversación)")
    }
}


async function obtenerYReproducirAudio(texto, index) {
    try {
        const response = await fetch(URL + '/audio', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify({ texto: texto, pausa: 'true'})
        });
        const data = await response.json();
        if (data.audio_base64) {
            console.log('Enviando audio al buffer de reproducción, tamaño:', data.audio_base64.length, 'índice:', index);
            addAudioClipToBuffer(data.audio_base64, index);
        }
    } catch (error) {
        console.error('Error al obtener el audio:', error);
    }
}



function updateResponseText(text) {
    // document.getElementById('responseText').innerText += text;
    textarea = document.getElementById('responseText')
    textarea.innerText += text;
    textarea.scrollTop = textarea.scrollHeight;
}



function obtenerYReproducirAll() {
    fetch(URL + '/all_conversation', {
        method: 'GET',
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
            createAudioAllPlayer(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}


function createAudioAllPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerAllContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

function obtenerYReproducirSaludo(texto) {
    //console.log('Obteniendo audio para:', texto, 'microsegundos:', Date.now() - milisegundosInicio);
    fetch(URL + '/audio', {

        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto, pausa: 'true'})
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
         //   console.log('Obteniendo audio, tamaño:', data.audio_base64.length, 'SALUDO', 'microsegundos:', Date.now()-milisegundosInicio);
            playAudio(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}





function addAudioClipToBuffer(base64Audio, index) {
    audioBuffer[index] = { audio: base64Audio};
    let audioPlayer = document.getElementById('audioPlayerContainer').querySelector('audio');
    //console.log('Despues de meter en Buffer, VIENDO si ejecuto play con indice:', index, 'pausa:',audioPlayer.paused, 'currentAudioIndex', currentAudioIndex,'microsegundos:', Date.now()-milisegundosInicio);
    if (audioPlayer.paused && currentAudioIndex == index) {
        playNextAudioClip();
    }
}

//function esObjetoVacio(obj) {
//  return Object.keys(obj).length === 0;
//}
function esObjetoVacio(obj) {
  try {
    // Intenta obtener las claves del objeto y verifica si su longitud es 0
    return Object.keys(obj).length === 0;
  } catch (error) {
    // Si ocurre un error (por ejemplo, si obj no es un objeto), devuelve true
    //console.error("Se produjo un error porque en esa posición del buffer está vacía: ", error);
    return true;
  }
}



function playNextAudioClip() {
    //console.log('Entrando en playNextAudioClip para ver si reproducimos audio con currentAudioIndex:', currentAudioIndex, 'microsegundos:', Date.now()-milisegundosInicio, 'esObjetoVacio:', esObjetoVacio(audioBuffer[currentAudioIndex]));
    if (!esObjetoVacio(audioBuffer[currentAudioIndex])) {
       // console.log('Iniciando variables para Reproducion audio concurrentAudioIndex:', currentAudioIndex, 'tamaño:', audioBuffer[currentAudioIndex].audio.length, 'microsegundos:', Date.now()-milisegundosInicio);
        const audioData = audioBuffer[currentAudioIndex];
        audioBuffer[currentAudioIndex] = {}; // Limpia el elemento actual
        currentAudioIndex++;

        let audioContainer = document.getElementById('audioPlayerContainer');
        audioContainer.innerHTML = ''; // Limpia el contenedor

        let audioSrc = `data:audio/wav;base64,${audioData.audio}`;
        let audioPlayer = document.createElement('audio');
        audioPlayer.src = audioSrc;
        audioPlayer.controls = true;
        audioPlayer.autoplay = true;

      //  console.log('Justo antes de APPENDCHILD Reproduciendo audio currentAudioIndex:', currentAudioIndex, 'microsegundos:', Date.now()-milisegundosInicio);
        audioContainer.appendChild(audioPlayer);

        audioPlayer.onended = playNextAudioClip;
    }
}

function playAudio(audioBase64) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    audioContainer.innerHTML = ''; // Limpia el contenedor

    let audioSrc = `data:audio/wav;base64,${audioBase64}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;

    //console.log('Reproduciendo saludo! Justo antes de APPENDCHILD', 'microsegundos:', Date.now()-milisegundosInicio);
    audioContainer.appendChild(audioPlayer);
}

function createAudioPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

obtenerYReproducirSaludo("Wellcom to Conversational Games!")



</script>


Cliente de juego de traducciones

In [ ]:

<head>
    <meta charset="UTF-8">
    <title>Translate Game</title>
  <style>


    body {
        font-family: Arial, sans-serif; /* Mejora la tipografía general */
    }



    #inicioForm {
        max-width: 1000px; /* Limita el ancho del formulario */
        margin: 20px auto; /* Centra el formulario */
        padding: 20px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); /* Añade un sombreado ligero */
    }

    #inicioForm div {
        margin-bottom: 15px; /* Añade más espacio entre los campos */
    }

    #inicioForm label {
        font-weight: bold; /* Hace que las etiquetas sean más notables */
        display: block; /* Asegura que la etiqueta esté encima del input */
        margin-bottom: 5px; /* Espacio entre la etiqueta y el campo */
    }

    #inicioForm select, #inicioForm textarea, #inicioForm button {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
    }

    #inicioForm select {
        cursor: pointer; /* Indica que es un elemento interactivo */
        font-size: 16px; /* Aumenta el tamaño del texto */
    }

    #inicioForm textarea {
        resize: vertical; /* Permite al usuario ajustar la altura verticalmente */
    }

    #inicioForm button {
        background-color: #007bff; /* Color de fondo */
        color: white; /* Color del texto */
        border: none; /* Elimina el borde */
        padding: 10px 15px; /* Añade relleno */
        font-size: 18px; /* Aumenta el tamaño del texto */
        cursor: pointer; /* Indica que es un elemento interactivo */
        border-radius: 5px; /* Bordes redondeados */
    }

    #inicioForm button:hover {
        background-color: #0056b3; /* Oscurece el botón al pasar el mouse */
    }

#audioPlayerContainer {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 20px; /* Espacio antes del botón de grabación */
}

#audioPlayerContainerSol {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 20px; /* Espacio antes del botón de grabación */
}

#recordButton {
    background-color: #f44336; /* Color rojo para la grabación */
    color: white;
    border: none;
    padding: 10px 0;
    font-size: 18px;
    border-radius: 5px;
    cursor: pointer;
}

#recordButton:hover {
    background-color: #d32f2f; /* Oscurece el botón al pasar el mouse */
}

/* Estilos para el área de texto y botón de envío */
/* Contenedor del área de texto y el botón */
div#textButtonContainer {
    display: flex; /* Establece el contenedor para usar flexbox */
    justify-content: space-between; /* Espacia los elementos uniformemente */
    align-items: center; /* Alinea los elementos verticalmente en el centro */
}

/* Área de texto */
#textInput {
    flex-grow: 1; /* Permite que el área de texto crezca para ocupar el espacio disponible */
    margin-right: 10px; /* Añade un margen a la derecha para separarlo del botón */
    border: 1px solid #ccc; /* Establece un borde sutil */
    border-radius: 5px; /* Bordes redondeados */
    padding: 8px; /* Añade padding interno */
}

/* Botón */
#sendTextButton {
    padding: 8px 15px; /* Ajusta el padding para dimensionar el botón */
    background-color: #4CAF50; /* Color de fondo */
    color: white; /* Color del texto */
    border: none; /* Elimina el borde */
    border-radius: 5px; /* Bordes redondeados */
    cursor: pointer; /* Cambia el cursor a mano al pasar sobre el botón */
}

#sendTextButton:hover {
    background-color: #388E3C; /* Oscurece el botón al pasar el mouse */
}


#responseText {
    height: 250px;
    margin-top: 20px;
    border: 1px solid #ddd;
    padding: 10px;
    overflow-y: auto; /* Asegura el desplazamiento vertical */
    background-color: #f9f9f9; /* Fondo claro para resaltar el área */
}


#nuevoGrupoSelect, #ejercicioNuevoGrupoSelect {
    width: 100%; /* Aprovecha todo el ancho disponible */
    padding: 8px; /* Añade un relleno para mayor comodidad */
    margin-top: 4px; /* Espacio mínimo superior para separación */
    cursor: pointer; /* Indica que es un elemento interactivo */
    font-size: 16px; /* Aumenta el tamaño del texto */
}

    
        /* Estilo básico para el botón de refrescar */
        .btn-refrescar {
            background-color: #f5f5f5;
            border: none;
            cursor: pointer;
            padding: 10px;
            border-radius: 5px;
            font-size: 16px;
        }
        /* Icono de refrescar utilizando una entidad HTML */
        .btn-refrescar:before {
            content: "\21BB"; /* Símbolo de flecha circular */
            margin-right: 5px;
        }

#campoEspañol, #campoIngles, #campoGrupo {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
}

</style>
</head>

<input type="file" id="fileInput" />
<button onclick="uploadFile()">Subir Fichero de Ejercicios</button>
<button onclick="descargarCSV()">Guardar cambios y Descargar</button>


<script>
function uploadFile() {
  const fileInput = document.getElementById('fileInput');
  if (!fileInput.files.length) {
    alert('Por favor, selecciona un archivo para subir.');
    return;
  }

  const file = fileInput.files[0];
  const reader = new FileReader();

  reader.onload = function(e) {
    // Una vez que el archivo ha sido leído, su contenido se almacena en la variable 'text'.
    const text = e.target.result;
    procesarCSV(text)

    // Aquí puedes hacer lo que necesites con el texto. Por ahora, solo lo mostraremos en una alerta.
    alert("Contenido del archivo cargado:\n" + text.substring(0, 200) + "..."); // Muestra los primeros 200 caracteres
  };

  reader.onerror = function() {
    alert('Hubo un error al leer el archivo');
  };

  // Leer el contenido del archivo como un string.
  reader.readAsText(file);
}


function generarTextoCSV(registros) {
    // Unir cada registro array a un string, separando las columnas con '#', y cada fila con un salto de línea
    return registros.map(fila => fila.join('#')).join('\n');
}

function descargarCSV() {
    const csvText = generarTextoCSV(registros);
    const blob = new Blob([csvText], { type: 'text/csv;charset=utf-8;' });
    const url = URL.createObjectURL(blob);

    // Crear un enlace para la descarga
    const downloadLink = document.createElement("a");
    downloadLink.href = url;
    downloadLink.download = "datos_exportados.csv";

    // Simular un clic en el enlace para iniciar la descarga y luego limpiar
    document.body.appendChild(downloadLink);
    downloadLink.click();
    document.body.removeChild(downloadLink);
    URL.revokeObjectURL(url);
    //guardarCambios();  //para guardar también en el servidor
}

</script>

<script>
let registros = []; // Almacena aquí todos los registros procesados

  // Función para cargar y procesar el archivo CSV
  function cargarCSVDesdeServidor() {
    fetch('https://javiergimenez.es/api/get-translations-file')
      .then(response => response.text())
      .then(text => procesarCSV(text))
      .catch(error => console.error('Error al cargar el archivo CSV:', error));
  }

  // Función para procesar el contenido del CSV y almacenar los registros
  function procesarCSV(csvText) {
    const filas = csvText.split('\n');
    registros = filas.map(fila => {
        const columnas = fila.replace(/\r/g, '').split('#');
        return columnas.map(columna => columna.trim()); 
    });
    const tipos = new Set();
    //tipos.add('All'); // Añadir una opción predeterminada
    tipos.add('repaso'); // Añadir una opción siempre existente

    registros.forEach(registro => {
      if (registro.length > 2) {
        registro[2].split('/').forEach(tipo => tipos.add(tipo));
      }
    });
    llenarSelector(Array.from(tipos), 'nuevoGrupoSelect'); // Actualiza para llenar el <select>
    llenarSelector(Array.from(tipos), 'ejercicioNuevoGrupoSelect'); // Actualiza para llenar el <select>
    //añadir al prinpio de la lista de tipos el valor 'All'
    tipos.add('All'); // Añadir una opción predeterminada
    llenarSelector(Array.from(tipos), 'ejercicios');

  }


//  function guardarCambios() {
//    // Supongamos que 'registros' contiene los datos modificados que queremos guardar
//    fetch('http://localhost:5500/save-translations-file', {
//        method: 'POST',
//        headers: {
//            'Content-Type': 'application/json',
//        },
//        body: JSON.stringify(registros)  // 'registros' debe ser el array de tus datos
//    })
//    .then(response => response.json())
//    .then(data => {
//        console.log(data.message); // Mensaje de éxito
//       alert(data.message);
//    })
//    .catch((error) => {
//        console.error('Error:', error); // Manejo de errores
//        alert('Error al guardar los cambios');
//    });
//}


    function llenarSelector(tipos, elementoID) {
        const elemento = document.getElementById(elementoID);
        elemento.innerHTML = ''; // Limpiar opciones existentes

        tipos.forEach(tipo => {
            const opcion = document.createElement('option');
            opcion.value = tipo;
            opcion.textContent = tipo; // Solo para el selector de ejercicios
            elemento.appendChild(opcion);
        });
    }

  

  // Función para manejar el cambio de selección y actualizar la variable ejercicio
  let ejercicio = 'All'; // Variable para almacenar el tipo seleccionado
  function actualizarEjercicio(valorSeleccionado) {
    ejercicio = valorSeleccionado;
    mostrarSiguiente()
    console.log('Ejercicio seleccionado:', ejercicio);
  }


//let indiceActual = -1; // Variable para almacenar el índice del registro actual

// Función para obtener un registro aleatorio que contenga el valor seleccionado como un string completo en la tercera columna
function obtenerRegistroAleatorio(tipoSeleccionado) {
  let registrosFiltrados;

  // Si el tipo seleccionado es "All", no filtra los registros
  if (tipoSeleccionado.trim() === "All") {
    registrosFiltrados = registros;
  } else {
    // Filtra los registros verificando que el tipo seleccionado, sin espacios en blanco,
    // sea exactamente uno de los tipos en la tercera columna después de hacer split por '/'
    registrosFiltrados = registros.filter(registro => {
      if (registro.length > 2) {
        // Divide la tercera columna por '/', elimina espacios en blanco y verifica si el tipo seleccionado está presente
        const tipos = registro[2].split('/').map(tipo => tipo.trim());
        return tipos.includes(tipoSeleccionado.trim());
      }
      return false;
    });
  }

  if (registrosFiltrados.length === 0) {
    return null; // O manejar de otra manera si no hay registros que coincidan
  }
  
  // Selecciona uno de los registros filtrados al azar y lo devuelve
  const indiceAleatorio = Math.floor(Math.random() * registrosFiltrados.length);
//   indiceActual = indiceAleatorio
  return registrosFiltrados[indiceAleatorio];
}


function eliminarRegistro(Pregistro) {
    // Encuentra el índice del registro para eliminar en 'registros' basado solo en los dos primeros campos
    español = Pregistro[0]
    ingles = Pregistro[1]
    let indice = registros.findIndex(registro => 
        registro[0] === español && registro[1] === ingles
    );

    // Verifica si se encontró el índice
    if (indice !== -1) {
        // Elimina el registro de 'registros' usando splice
        registros.splice(indice, 1);
        console.log('Registro eliminado correctamente.');
    } else {
        console.log('Registro no encontrado.');
    }
}





// Función para añadir un grupo a la tercera columna del registro especificado por el índice
function añadirGrupo(registro, grupo) {
  // Verifica que el índice esté dentro del rango de los registros cargados
  if (registro) {
    // Divide la tercera columna por '/', elimina espacios en blanco de los extremos, y verifica si el grupo ya está presente
    let grupos = registro[2].split('/').map(g => g.trim());
    if (!grupos.includes(grupo)) {
      grupos.push(grupo); // Añade el nuevo grupo
      registro[2] = grupos.join('/'); // Reconstruye la tercera columna y actualiza el registro
      return true; // Indica que el grupo fue añadido exitosamente
    } else {
      return false; // Indica que el grupo ya estaba presente
    }
  } else {
    console.error('Índice fuera de rango.');
    return false; // También retorna false si el índice está fuera de rango
  }
}

// Función para eliminar un grupo de la tercera columna del registro especificado por el índice
function eliminaGrupo(registro, grupo) {
  // Verifica que el índice esté dentro del rango de los registros cargados
  if (registro) {
    // Divide la tercera columna por '/', elimina espacios en blanco de los extremos, y verifica si el grupo existe
    let grupos = registro[2].split('/').map(grupo => grupo.trim());
    if (grupos.includes(grupo)) {
      grupos = grupos.filter(g => g !== grupo); // Elimina el grupo
      registro[2] = grupos.join('/'); // Reconstruye la tercera columna y actualiza el registro
      return true; // Indica que el grupo fue eliminado exitosamente
    } else {
      return false; // Indica que el grupo no se encontró
    }
  } else {
    console.error('Índice fuera de rango.');
    return false; // También retorna false si el índice está fuera de rango
  }
}



function arrayToCSVString(arrayDeArrays) {
    // Mapea cada sub-array a un string, uniendo los campos con '#'
    const lineas = arrayDeArrays.map(registro => registro.join('#'));
    // Une todas las líneas con saltos de línea para formar el string completo del CSV
    const csvString = lineas.join('\n');
    return csvString;
}


  // Llama a cargarCSVDesdeServidor al cargar la página
 // window.addEventListener('load', cargarCSVDesdeServidor);



</script>


<form id="inicioForm">
    <div>
        <label for="enunciado">Traduce esto:</label><br>
        <textarea id="enunciado" name="enunciado" rows="1" cols="100"></textarea>
    </div>
    <div>
        <label for="ejercicios">Juegos:</label> <button class="btn-refrescar" onclick="procesarCSV(arrayToCSVString(registros))">Refrescar</button>
<br>
        <select id="ejercicios" onchange="actualizarEjercicio(this.value)">
        <!-- Las opciones se llenarán dinámicamente -->
        </select>
       
    </div>
    <button type="submit">Siguiente!</button>

</form>

<div id="manejoGrupos">
    <!-- Primera fila con botones para añadir a repaso y eliminar del grupo -->
    <div>
        <button id="btnAñadirRepaso">Añadir a repaso</button>
        <button id="btnEliminarRepaso">Eliminar de repaso</button>
        <button id="btnEliminarRegistro">Eliminar registro</button>
        <button id="btnGuardarCambios">Guardar Cambios</button>
    </div>
    
    <p> AÑADIR O ELIMINAR ESTE EJERCICIO DE UN GRUPO: </p>
    <div>
        <select id="nuevoGrupoSelect" onchange="seleccionarGrupo()">
            <!-- Las opciones se llenarán dinámicamente -->
        </select>
        <input type="text" id="nuevoGrupoInput" placeholder="Nuevo grupo o seleccione">
        <button id="btnAñadirNuevoGrupo">Añadir al grupo</button>
        <button id="btnEliminarDelGrupo">Eliminar del grupo</button>

    </div>
</div>

<script>


let registroActual

//document.getElementById('btnEliminarRegistro').addEventListener('click', eliminarRegistro(registroActual));
document.getElementById('btnEliminarRegistro').addEventListener('click', function() {
    eliminarRegistro(registroActual);
});

//document.getElementById('btnGuardarCambios').addEventListener('click', guardarCambios);
//document.getElementById('btnGuardarCambios').addEventListener('click', function() {
//    guardarCambios();
//});



document.getElementById('btnAñadirRepaso').addEventListener('click', function() {
    if (añadirGrupo(registroActual, 'repaso')) {
        alert('Añadido a repaso'); // Si el grupo fue añadido exitosamente
    } else {
        // Si el registro ya estaba en el grupo o hubo algún otro error
        alert('El registro ya está en el grupo repaso o no se pudo añadir');
    }
});

document.getElementById('btnEliminarRepaso').addEventListener('click', function() {
    // Utiliza la función eliminaGrupo para intentar eliminar el registro actual del grupo "repaso"
    if (eliminaGrupo(registroActual, 'repaso')) {
        alert('Eliminado del grupo repaso'); // Si el grupo fue eliminado exitosamente
    } else {
        // Si el registro no estaba en el grupo o hubo algún otro error
        alert('No se ha eliminado del grupo porque, en realidad, no se encontraba en el grupo repaso');
    }
});


document.getElementById('btnEliminarDelGrupo').addEventListener('click', function() {
    // Asume que tienes un selector definido en alguna parte de tu HTML para elegir el grupo
    let grupoSeleccionado = document.getElementById('nuevoGrupoInput').value; // Asegúrate de que el ID coincida
    if (eliminaGrupo(registroActual, grupoSeleccionado)) {
        alert('Eliminado del grupo ' + grupoSeleccionado); // Opcional: feedback al usuario
    } else {
        // Muestra un mensaje de error si el registro no se encontraba en el grupo
        alert(`No se ha eliminado del grupo porque, en realidad, no se encontraba en el grupo ${grupoSeleccionado}`);
    }
});

document.getElementById('btnAñadirNuevoGrupo').addEventListener('click', function() {
    let nuevoGrupo = document.getElementById('nuevoGrupoInput').value.trim();
    if (nuevoGrupo) {
        // La función añadirGrupo ahora devuelve true si el grupo fue añadido con éxito, y false si ya existía
        if (añadirGrupo(registroActual, nuevoGrupo)) {
            alert('Añadido al grupo ' + nuevoGrupo); // Si el grupo fue añadido exitosamente
        } else {
            // Si el registro ya estaba en el grupo
            alert(`El registro ya se encuentra en el grupo ${nuevoGrupo}, por lo que no se ha añadido de nuevo.`);
        }
    } else {
        alert('Por favor, introduce un nombre de grupo válido.'); // Si el input está vacío o es inválido
    }
});


function mostrarSiguiente() {
    // obtener un registro aleatorio
    const enunciadoTextArea = document.getElementById('enunciado');
    const ejercicio = document.getElementById('ejercicios').value;
    registroActual = obtenerRegistroAleatorio(ejercicio);
    enunciadoTextArea.value = registroActual[0];
 
    document.getElementById('responseText').innerText = registroActual[0];
}

function seleccionarGrupo() {
    const select = document.getElementById('nuevoGrupoSelect');
    const input = document.getElementById('nuevoGrupoInput');
    if (select.value) {
        input.value = select.value; // Transfiere el valor seleccionado al input
    }
}

function seleccionarNuevoEjercicioGrupo() {
    const select = document.getElementById('ejercicioNuevoGrupoSelect');
    const input = document.getElementById('campoGrupo');
    if (select.value) {
        input.value = select.value; // Transfiere el valor seleccionado al input
    }
}


document.getElementById('inicioForm').addEventListener('submit', function(e) {
    // Prevenir el comportamiento predeterminado del formulario
    e.preventDefault();

    // obtener un registro aleatorio
    mostrarSiguiente()
  
});

</script>


<div id="audioPlayerContainer"></div> <div id="audioPlayerContainerSol"></div>
<button id="recordButton" style="width: 100%; height: 50px;">Pulsa para grabar/detener</button>


<!-- Estilos para textarea y botón de envío -->
<div id="textButtonContainer" style="margin-top: 10px;">
    <textarea id="textInput" placeholder="Escribe tu texto aquí" rows="4"></textarea>
    <button id="sendTextButton">Enviar Texto</button>
</div>



<div id="responseText" style="height: 250px; margin-top: 20px; border: 1px solid #ddd; padding: 10px; overflow-y:auto;"></div>


<p> AÑADIR UN NUEVO EJERCICIO: </p>
<div id="nuevoEjercicio">
    <div>
        <label for="campoEspañol">Español:</label>
        <input type="text" id="campoEspañol" name="campoEspañol">
    </div>
    <div>
        <label for="campoIngles">Inglés:</label>
        <input type="text" id="campoIngles" name="campoIngles">
    </div>
    <div>
        <label for="campoGrupo">Grupo:</label>
         <select id="ejercicioNuevoGrupoSelect" onchange="seleccionarNuevoEjercicioGrupo()">
            <!-- Las opciones se llenarán dinámicamente -->
        </select>        
        <input type="text" id="campoGrupo" name="campoGrupo">
        
    </div>
    <button id="btnCrearNuevoEjercicio">Crear nuevo ejercicio</button>
</div>


<script>

document.getElementById('btnCrearNuevoEjercicio').addEventListener('click', function() {
    // Obtiene los valores de los campos de texto
    const español = document.getElementById('campoEspañol').value.trim();
    const ingles = document.getElementById('campoIngles').value.trim();
    
    const grupo = document.getElementById('campoGrupo').value.trim();


    // Valida que los campos no estén vacíos (ajusta según necesidades)
    if (español && ingles && grupo) {
        // Inserta los nuevos valores al final del array 'registros'
        registros.push([español, ingles, grupo]);
        console.log('Nuevo ejercicio creado:', español, ingles, grupo);
        console.log('Registros actualizados:', registros);

        // Opcional: Limpia los campos después de la inserción
        document.getElementById('campoEspañol').value = '';
        document.getElementById('campoIngles').value = '';
        document.getElementById('campoGrupo').value = '';
    } else {
        // Opcional: Mensaje de error si algún campo está vacío
        console.error('Todos los campos son requeridos para crear un nuevo ejercicio.');
        alert('Por favor, rellena todos los campos.');
    }
});


let recordButton = document.getElementById("recordButton");
let chunks = [];
let mediaRecorder;
let isRecording = false;

navigator.mediaDevices.getUserMedia({ audio: true })
.then(stream => {
    mediaRecorder = new MediaRecorder(stream);
    mediaRecorder.ondataavailable = event => {
        chunks.push(event.data);
    };
    mediaRecorder.onstop = () => {
        console.log('mediaRecorder Detenido!!');
        let blob = new Blob(chunks, { 'type': 'audio/ogg; codecs=opus' });
        enviarAudioAlServidor(blob);
        chunks = [];
    };
});

recordButton.onclick = () => {
    if (!isRecording) {
        mediaRecorder.start();
        isRecording = true;
        recordButton.textContent = 'Grabando...';
    } else {
        mediaRecorder.stop();
        isRecording = false;
        recordButton.textContent = 'Pulsa para grabar/detener';
    }
};



function enviarTexto(texto) {
    updateResponseText(" ---> " + texto + " (sol: " + registroActual[1] + ")" + "\n");
    obtenerYReproducir(texto, false);
    obtenerYReproducir(registroActual[1]);
}




function enviarAudioAlServidor(blob) {
    let formData = new FormData();
    formData.append('file', blob, 'grabacion.ogg');

    fetch('https://javiergimenez.es/api/only_transcribe', {
        method: 'POST',
        body: formData,
    })
    .then(response => response.json())
    .then(data => {
        updateResponseText(" ---> " + data.entrada + " (sol: " + registroActual[1] + ")" + "\n");
     

    })
    .catch(error => {
        console.error('Error al enviar el audio:', error);
    });

    obtenerYReproducir(registroActual[1]);

    createAudioPlayerFromBlob(blob)
}



// Funcionalidad para enviar texto al servidor y limpiar el textarea
document.getElementById("sendTextButton").onclick = () => {
    let textInput = document.getElementById("textInput");
    let texto = textInput.value;
    if (texto) {
        enviarTexto(texto);
        textInput.value = ''; // Limpiar el textarea después de enviar
    }
};




function updateResponseText(text) {
    // document.getElementById('responseText').innerText += text;
    textarea = document.getElementById('responseText')
    textarea.innerText += text;
    textarea.scrollTop = textarea.scrollHeight;
}



function obtenerYReproducir(texto, sol=true) {
    //relleno para que no tenga problemas el generador de voz.
    //if (texto.length < 20) { texto += " . (This phrase was too short)";}
    //else if (texto.length < 25) { texto += " . .";}
    console.log('Obteniendo audio para:', texto);
    fetch('https://javiergimenez.es/api/audio', {

        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto, pausa: 'false' })
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {            
            playAudio(data.audio_base64, sol);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}




function playAudio(audioBase64, sol=true) {
    let audioContainer=""
    if (sol) {
        audioContainer = document.getElementById('audioPlayerContainerSol');
    } else {
        audioContainer = document.getElementById('audioPlayerContainer');
    }
    audioContainer.innerHTML = ''; // Limpia el contenedor

    let audioSrc = `data:audio/wav;base64,${audioBase64}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = false;

    console.log('creando audio para solución.');
    audioContainer.appendChild(audioPlayer);
}

function createAudioPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

function createAudioPlayerFromBlob(blob) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    // Crea una URL de objeto para el Blob
    let audioSrc = URL.createObjectURL(blob);
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = false;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

cargarCSVDesdeServidor()

</script>
